In [3]:
import numpy as np
import pandas as pd
import numpy as np
import scipy.optimize as opt

In [4]:
## COMMON PARAMETERS
R = 20
L = 5.0
β = 20
γbase = 10
u0 = 4.0 / 60
ω = 1200
f = lambda z: ω - fz * z 
seats = 25
crowding = 0.1
γ = lambda k: γbase + crowding*np.maximum(0,k-seats)**2

In [11]:
# WITH FARE
cb = 160
fz = 100
ZPE = .5
μ = lambda q: u0 / (1 - q * δ)

δ = 3 / 3600
Γ = lambda q: γ(q*L*μ(q))
T = lambda Q, B: (β * R / 2 / B + Γ(Q/B) * L) * μ(Q / B) 
Z = lambda p,Q,B: p + T(Q,B) + ZPE

def maxS():
    res = opt.minimize(
            lambda x: -(x[2] ** 2 / (2 * fz) + x[2] * x[0] - x[1] * cb),
            bounds=((0, 20), (1, 50), (10, ω)),
            x0=(1, 5, ω/2),
            method="SLSQP",
            constraints=({
                "type": "ineq", "fun": lambda x: 1 - δ*x[2]/x[1],
                "type": "eq", "fun": lambda x: f(Z(x[0],x[2],x[1])) - x[2],
            }),
    )
    p, B, Q = res.x
    z = Z(p,Q,B)
    return (
        p,
        B,
        Q,
        z,
        -res.fun,
        p * Q - cb * B,
        Q * μ(Q / B) * L / B,
        μ(Q / B) * L * 60,
        Q / B,
    )



In [18]:
cb2 = cb - 20
δ2 = 1 / 3600
μ2 = lambda q: u0 /(1 - q * δ2)
Γ2 = lambda q: γ(q*μ2(q)*L)
T2 = lambda Q, B: (β * R / 2 / B + Γ2(Q/B) * L) * μ2(Q / B)
def maxS2():
    res = opt.minimize(
        lambda x: -(x[1] ** 2 / (2 * fz) - x[0] * cb2),
        method="SLSQP",
        bounds=((1, 20), (1, ω)),
        x0=(1, ω/3),
            constraints=({
                "type": "ineq", "fun": lambda x: np.array([
                                                            1 - δ2*x[1]/x[0],
                                                            f(T2(x[1], x[0])) - x[1],
#                                                             1003. - x[0]*cb2 
                                                          ]),
            })
    )
    B, Q = res.x
    z = T2(Q, B)
    return (
        0,
        B,
        Q,
        z,
        -res.fun,
        -cb2 * B,
        Q * μ2(Q / B) * L / B,
        μ2(Q / B) * L * 60,
        Q / B,
    )

In [16]:
## DATAFRAME OF BOTH RESULTS

df = pd.DataFrame(
    np.array([maxS(), maxS2()]),
    columns=["p", "B", "Q", "z", "S", r"$\pi$", "k", "t", "q"],
)
df.round(3)

,p,B,Q,z,S,$\pi$,k,t,q
0,0.498,8.046,569.812,6.302,619.921,-1003.506,25.088,21.254,70.821
1,0.000,9.662,718.408,4.816,1227.866,-1352.683,25.307,20.422,74.354
